In [7]:
import tensorflow as tf
import numpy as np

In [205]:
class SearchAreaEnvironment:
    
    def __init__(self, k_values_map, agents_start_positions):
        
        unit_vector_count = 5
        self.KVM = k_values_map #load all the k-values
        self.width = k_values_map.shape[1] #Width of the map
        self.height = k_values_map.shape[0] #height of the map
        self.dKVM = np.zeros((self.height, self.width)) #initialize discovered KVM.
        agents_start_positions = np.array(agents_start_positions)
        self.agents_start_pos = np.array(agents_start_positions, copy=True) #save starting positions
        self.agents_pos = np.array(agents_start_positions, copy=True) #vector of 1x2 containing information of the position of the agents
        self.agents_count = agents_start_positions.shape[0] #amount of agents
        self.path_matrices = np.array([[[0,0] for _ in range(unit_vector_count)] for _ in range(self.agents_count)]) #path matrices for each agent
        
        
        
        
        
        
    def reset(self):
        
        self.dKVM = np.zeros((self.height, self.width)) #the whole map is undiscovered
        self.agents_pos = np.array(self.agents_start_pos, copy=True) #set current positions to start positions
        
        
        
        
        
        
    def get_pos(self, label_agent):
        
        self.x, self.y = self.agents_pos[label_agent-1] #get current state of any drone
        
        return [self.x, self.y]
    
    
    
    
    
    
    def get_k_value(self, x, y):
        
        regional_k_value = self.dKVM[y, x]
        
        return regional_k_value
    
    
    
    
    
        
    def get_dKVM(self): #update dKVM
        
        for i in range(self.agents_count):
            
            p = self.agents_pos[i]
            self.dKVM[p[1]][p[0]] = self.KVM[p[1]][p[0]]  #Update dKVM base on the location of every agents
            
            
            
            
            
            
    def new_path_matrices_update(self, path_matrcies):
        
        self.path_matrices = path_matrcies
        
        
        
        
        
        
    def difference_in_k(self):
        
        difference_k_array = np.array([])
        
        for i in range(self.agents_count):
            
            current_pos = get_pos(i+1)
            future_pos = self.matrices[i][0]
            
            current_k = self.get_k_value(current_pos[0], current_pos[1])
            future_k = self.get_k_value(future_pos[0], future_pos[0])
            
            difference_in_k = future_k - current_k
            
            difference_k_array.append(difference_in_k)
            
        return difference_k_array
            
            
            
            
        
        
    def take_action(self): #get path plan, move agents
        
        for i in range(self.agents_count): #update position of every agents
            
            pos_update = self.path_matrices[i][0]  #next step move
            current_pos = self.get_pos(i+1) #current position
            new_pos = np.array([pos_update[0] + current_pos[0], pos_update[1] + current_pos[1]]) #sum
            
            self.agents_pos[i] = new_pos #update position
            
            self.process_old_path_matrices() #process the matrices to be ready to put into the neural network
            
            
            
            
            
    def process_old_path_matrices(self): #for old paths, remove the complete vector, add a new empty vector
        
        processed_path_matrices = []
        
        for agents in self.path_matrices:
            processed_path = list(agents[1:])  # Remove the first element of the second dimension
            processed_path.append([0, 0])  # Add [0, 0] at the end
            processed_path_matrices.append(processed_path)
            
        self.path_matrices = np.array(processed_path_matrices)

            
            
            
            
        
        

In [207]:
k_values_map = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

agents_start_positions = np.array([
    [0, 0],  # Agent 1 starts at position (0, 0)
    [2, 2]   # Agent 2 starts at position (2, 2)
])

new_path_matrices = np.array(
    [[[1, 0], [-1, 0], [0, 0], [0, 0], [0, 0]], [[-1, 0], [1, 0], [0, 0], [0, 0], [0, 0]]]
)

s = SearchAreaEnvironment(k_values_map, agents_start_positions)

new_path_matrices = np.array(
    [[[1, 1], [0, 0], [0, 0], [0, 0], [0, 0]], [[-1, -2], [0, 0], [0, 0], [0, 0], [0, 0]]]
)

s.new_path_matrices_update(new_path_matrices)

print(s.agents_pos)
print(s.agents_start_pos)

s.take_action()

print(s.agents_pos)
s.get_dKVM()
print(s.dKVM)

print(s.agents_start_pos)

s.new_path_matrices_update(new_path_matrices)

print(s.agents_pos)
print(s.agents_start_pos)

s.take_action()

print(s.agents_pos)
s.get_dKVM()
print(s.dKVM)

print(s.agents_start_pos)


s.reset()
print("new \n\n\n\n\n")


s.new_path_matrices_update(new_path_matrices)

print(s.agents_pos)
print(s.agents_start_pos)

s.take_action()

print(s.agents_pos)
s.get_dKVM()
print(s.dKVM)

print(s.agents_start_pos)




[[0 0]
 [2 2]]
[[0 0]
 [2 2]]
[[1 1]
 [2 2]]
[[0. 0. 0.]
 [0. 5. 0.]
 [0. 0. 9.]]
[[0 0]
 [2 2]]
[[1 1]
 [2 2]]
[[0 0]
 [2 2]]
[[2 2]
 [2 2]]
[[0. 0. 0.]
 [0. 5. 0.]
 [0. 0. 9.]]
[[0 0]
 [2 2]]
new 





[[0 0]
 [2 2]]
[[0 0]
 [2 2]]
[[1 1]
 [2 2]]
[[0. 0. 0.]
 [0. 5. 0.]
 [0. 0. 9.]]
[[0 0]
 [2 2]]


In [ ]:
'''get_dKVM -> agents_pos -> neural network (input, dKVM, self.path_matricies, agents_pos ) -> 
output(new_path_matricies) -> new_path_matrcies_update -> take_action
'''